In [1]:
import os

In [48]:
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")



In [3]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.schema import HumanMessage, SystemMessage

In [20]:
llm=ChatOpenAI(temperature=0)

In [7]:
message=[
    SystemMessage(
        content="The user will input the year and you will get baseball world series champion "
    ),
    HumanMessage(
        content="2023"
    )
]

In [8]:
llm.invoke(message)

AIMessage(content='The 2023 World Series champion is not yet known as it is a future event.', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 28, 'total_tokens': 46}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [9]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [10]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [15]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
wikitool=WikipediaQueryRun(api_wrapper=api_wrapper)
tools=[wikitool]

In [16]:
agent=create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)

In [17]:
agent_executor=AgentExecutor(agent=agent,tools=tools)

In [44]:
agent_executor.invoke({"input": "what is Wikipedia news of india now"})

{'input': 'what is Wikipedia news of india now',
 'output': "I couldn't find specific news updates about India on Wikipedia. Would you like me to look up news updates from a different source?"}

In [37]:
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import Tool
from langchain.tools import BaseTool

search=DuckDuckGoSearchRun()

search_tool=Tool(
        name='search',
        func=search.run,
        description="Tool to answer about current events. You should ask targeted questions"
    )


In [41]:
## Agent creation
from langchain.agents import initialize_agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory


tools=[search_tool, description_of_Kamalesh369_tool]

## Conversational agent memory
memory= ConversationBufferWindowMemory(
    memoryview='chat_history',
    k=3,
    return_messages=True
)

## Agent
conversational_agent=initialize_agent(
    agent='conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)

In [42]:
conversational_agent({"input":"What is the time at India now?"})



> Entering new AgentExecutor chain...


ValueError: Missing some input keys: {'chat_history'}

In [39]:
search.run("time at India?")

RatelimitException: https://links.duckduckgo.com/d.js?q=time+at+India%3F&kl=wt-wt&l=wt-wt&vqd=4-145324765630198312189260572860599974425&bing_market=wt-wt&a=ftsa&ex=-1&df=y&s=0 202

c:\Custom_Tools_and_Agents_Langchain\venv\Lib\site-packages\curl_cffi\aio.py:205: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


In [49]:
# from langchain_community.tools.tavily_search import TavilySearchResults
# TavilySearch_tool = TavilySearchResults()

# # tool.invoke({"query": "latest news in India on 28th March 2024"})





[{'url': 'https://timesofindia.indiatimes.com/topic/march-28/news',
  'content': '15 hours ago — 15 hours agoOn march 28, 2024, Capricorn delves into introspection, inner growth, emotional bonds, strategic realignment, and well-being practices, fostering personal\xa0...'},
 {'url': 'https://www.narendramodi.in/social-media-corner-28th-march-twenty-twenty-four-580898',
  'content': '5 hours ago — 5 hours agoUnion Minister Anurag Singh Thakur graced the Viksit Bharat Ambassador Meet-up held in Jammu on Wednesday, March 27, 2024. Addressing the\xa0...'}]

In [78]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI

## Custom tools
def description_of_Kamalesh369(input=""):
    return 'Kamalesh369 is a Data-scientist who has extensive knowledge on ML-OPS, Data Engineering, Cloud computing and GenAI'

description_of_Kamalesh369_tool=Tool(
    name='Description_of_Kamalesh369',
    func=description_of_Kamalesh369,
    description="Useful tool to answer questions about Kamalesh369. Input should be Kamalesh369"
)

instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)
llm = ChatOpenAI(temperature=0)
tavily_tool = TavilySearchResults()
tools = [description_of_Kamalesh369_tool,tavily_tool]
# agent = create_openai_functions_agent(llm, tools, prompt)
# agent_executor = AgentExecutor(
#     agent=agent,
#     tools=tools,
#     verbose=True,
# )

In [85]:
## Agent creation
from langchain.agents import initialize_agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory


# tools=[search_tool, description_of_Kamalesh369_tool]
tools = [description_of_Kamalesh369_tool,tavily_tool]

## Conversational agent memory
memory= ConversationBufferWindowMemory(
    memoryview='chat_history',
    k=3,
    return_messages=True
)

## Agent
conversational_agent=initialize_agent(
    agent='conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    # memory=memory
)

In [86]:
conversational_agent({"input":"what do you know about Kamalesh369?",
                      "chat_history": [
                        HumanMessage(content="hi! my name is Kamalesh"),
                        SystemMessage(content="Hello Kamalesh! How can I assist you today?"),
        ],})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Description_of_Kamalesh369
Action Input: Kamalesh369
Observation: Kamalesh369 is a Data-scientist who has extensive knowledge on ML-OPS, Data Engineering, Cloud computing and GenAI
Thought:Do I need to use a tool? No
AI: Kamalesh369 is a Data-scientist who has extensive knowledge on ML-OPS, Data Engineering, Cloud computing, and GenAI.

> Finished chain.


{'input': 'what do you know about Kamalesh369?',
 'chat_history': [HumanMessage(content='hi! my name is Kamalesh'),
  SystemMessage(content='Hello Kamalesh! How can I assist you today?')],
 'output': 'Kamalesh369 is a Data-scientist who has extensive knowledge on ML-OPS, Data Engineering, Cloud computing, and GenAI.'}